# GBasis Tutorial - Basis Sets

# 1 Basis Sets

In [gbasis](https://gbasis.readthedocs.io/en/latest/), a basis set is defined as a list of [`GeneralizedContractionShell`](add_link) objects, each one representing a linear combination of primitives and containing all the information defining this combination (i.e the angular momentum of the shell, the center of the shell, the exponents of the primitives, the contraction coefficients, and the normalization coefficients).

This notebook showcases the different ways to define a basis set for a molecule and how to transform them.

## 1.1 Loading Basis Sets

Basis set information is usually available in text format, such as from the [Basis Set Exchange](https://www.basissetexchange.org/). In [gbasis](https://gbasis.readthedocs.io/en/latest/), two of the most popular formats are supported:
  - Gaussian94 (.gbs)
  - NWChem (.nw).

The following example shows how to load a basis set from a Gaussian94  or a NWChem file.

In [ ]:
import numpy as np
from gbasis.parsers import parse_gbs, parse_nwchem

# use H2 molecule as an example system
atoms = ["H", "H"]
coords = np.array([[0, 0, 0], [0, 0, 1]])

print("Checking that the Gaussian94 and NWChem loaded basis sets are the same:")

# load hydrogen def2-SVP basis set information with the Gaussian94 format
gbs_basis_dict = parse_gbs("hydrogen_def2-svp.1.gbs")

# basis set information is stored as
# {'Atom Symbol': [(Angular Momentum, [Exponents], [Coefficients]), ...]}
print("def2-SVP Basis Set Loaded from Gaussian94 Format:")
for atom in gbs_basis_dict:
    print(f"Atom: {atom}")
    print(f"   Number of shells: {len(gbs_basis_dict[atom])}")
    for shell_num, shell in enumerate(gbs_basis_dict[atom]):
        print(f"   Shell {shell_num} has angular momentum {shell[0]}")
        print(f"   Shell {shell_num} has exponents {shell[1]}")
        print(f"   Shell {shell_num} has coefficients {shell[2].flatten()}")
print()

# load hydrogen def2-SVP basis set information with the NWChem format
nw_basis_dict = parse_nwchem("hydrogen_def2-svp.1.nw")

print("Checking that the Gaussian94 and NWChem loaded basis sets are the same:")
print("Both loaded basis sets have the same:")
for atom in gbs_basis_dict:
    print(f"Atom: {atom}: {atom in nw_basis_dict}")
    print(f"Number of shells: {len(gbs_basis_dict[atom]) == len(nw_basis_dict[atom])}")
    # check that the angular momentum, exponents, and coefficients are the same for each shell
    contractions_pair = enumerate(zip(gbs_basis_dict[atom], nw_basis_dict[atom]))
    for shell_num, (gb_shell, nw_shell) in contractions_pair:
        print(
            f"Shell {shell_num} has the same angular momentum: {gb_shell[0] == nw_shell[0]}"
        )
        print(
            f"Shell {shell_num} has the same exponents: {np.allclose(gb_shell[1], nw_shell[1])}"
        )
        print(
            f"Shell {shell_num} has the same coefficients: {np.allclose(gb_shell[2], nw_shell[2])}"
        )

Once the data for a basis set for a list of atoms is loaded, gbasis allows to build the contraction shells for a molecule. This is done by the [`make_contraction_shells`](add_link) function. The following example shows how to use this function to build the contraction shells for $\mathrm{H}_{2}$.

In [ ]:
from gbasis.parsers import make_contractions

# use H2 molecule as an example system
atoms = ["H", "H"]
coords = np.array([[0, 0, 0], [0, 0, 1]])

# make contractions for the hydrogen def2-SVP basis set
gbs_basis = make_contractions(gbs_basis_dict, atoms, coords)

print("Number of contracted basis functions:", len(gbs_basis))  # 3 for each H atom
print("Showing first three contraction shells:")
for i, basis in enumerate(gbs_basis[:3]):
    print(f"Contraction shell #{i}")
    print(f"   Center: {basis.coord}")
    print(f"   Angular momentum: {basis.angmom}")
    print(f"   Primitive coefficients {basis.coeffs.T}")
    print(f"   Primitive exponents {basis.exps}")
    print(f"   Primitive normalization constant {basis.norm_cont}")

## 1.2 Interfacing with other packages

[gbasis](https://gbasis.readthedocs.io/en/latest/) has been designed to be easily interfaced the [iodata](https://iodata.readthedocs.io/en/latest/) package which handles the inputs and outputs for different quantum chemistry formats, such as Gaussian formatted checkpoint files (.fchk) and AIM wavefunction files (.wfn and .wfx). Additionally, [gbasis](https://gbasis.readthedocs.io/en/latest/) can be directly interfaced with [pySCF](https://pyscf.org/), a popular Python package for quantum chemistry calculations.

The following example shows how to load a basis set from a Gaussian formatted checkpoint using the [`iodata`](https://iodata.readthedocs.io/en/latest/) package, the same approach can be usef dor loading the basis information of other formats supported by [`iodata`](https://iodata.readthedocs.io/en/latest/).

In [2]:
from iodata import load_one
from pyscf import gto
from gbasis.wrappers import from_iodata, from_pyscf

# build a basis from an fchk file (water at uwB97XD/def2-TZVPD LOT) using IOData
iodata_mol = load_one("water.fchk")
iodata_basis = from_iodata(iodata_mol)

# build an STO-3G basis for water using PySCF
pyscf_mol = gto.Mole()
pyscf_mol.build(
    atom = "O 0 0 0; H 0 1 0; H 0 0 1",
    basis = "sto-3g"
)
pyscf_basis = from_pyscf(pyscf_mol)

## 1.3 Types of Coordinate Systems Used by Basis Functions

In `gbasis`, the user can provide the coordinate system used by each shell of generalized contractions that gets stored in the basis list. The `make_contractions` function is useful for instantiating a basis for a system of atoms based on the `basis_dict` output of the parsers from `gbasis.parsers`. For a given call of `make_contractions`, one can pass either a string ("cartesian" or "spherical") or a list of strings to specify `coord_types`. If a string is passed, all contractions for all atoms are treated as having the same `coord_type`. 
If different shells correspond to different coordinate systems, then a list of the same length as the basis must be provided with each entry being "spherical" or "cartesian" to specify the coordinate system
of the corresponding shell. For example, if atom 1 contributes 8 contractions and atom 2 contributes 8 contractions, the `coord_types` list should be of length 16, where the first 8 elements correspond to contractions for atom 1 and the last 8 elements correspond to contractions for atom 2. These possibilities are illustrated in the code cell below using the Kr<sub>2</sub> molecule as an example:

In [3]:
# load STO-6G basis set information
basis_dict = parse_nwchem("data_sto6g.nwchem")

# create basis for Kr2 using cartesian coordinates for all generalized contractions
cartesian_basis = make_contractions(basis_dict, 
                                    ["Kr", "Kr"], 
                                    np.array([[0, 0, 0], [1, 0, 0]]),
                                    coord_types="cartesian")
print([shell.coord_type for shell in cartesian_basis])
print()

# create basis for Kr2 using spherical coordinates for all generalized contractions
spherical_basis = make_contractions(basis_dict, 
                                    ["Kr", "Kr"], 
                                    np.array([[0, 0, 0], [1, 0, 0]]),
                                    coord_types="spherical")
print([shell.coord_type for shell in spherical_basis])
print()

# create basis for Kr2 using spherical coordinates for all generalized contractions
# except the last one, which is specified to be cartesian
mixed_basis = make_contractions(basis_dict, 
                                ["Kr", "Kr"], 
                                np.array([[0, 0, 0], [1, 0, 0]]),
                                coord_types=["spherical"] * 15 + ["cartesian"])
print([shell.coord_type for shell in mixed_basis])

['cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian']

['spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical']

['spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'cartesian']


## 1.4 Linear Transformations of Basis Functions

In `gbasis`, the user can linearly transform the basis functions before computing the desired properties.
All of the higher level functions have the keyword argument `transform` to specify the matrix that
transforms the basis set. The transformation is applied to the left, i.e.,

$$\begin{equation}
 \psi_i = \sum_j T_{ij} \phi_j
\end{equation}$$

where {$\phi_{j}$} is the basis set before transformation and {$\psi_{i}$} is the basis function after transformation.
The number of basis functions depends on the coordinate systems specified for each shell. 

Examples involving such transformations are provided in the tutorial notebooks on evaluations and integrals wherever molecular rather than atomic orbitals are involved in calculations.